In [24]:
import os
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings as CommunityHuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains import LLMChain

# Carrega as variáveis de ambiente do .env
load_dotenv()

# Configura o LLM da Groq e o modelo de embeddings do Hugging Face
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.7)
llm_router = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Conecta-se ao Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "travel-assistant-index"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Função auxiliar para formatar os documentos de contexto
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
itinerary_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens especializado em criar roteiros detalhados.
    Use o contexto abaixo para gerar um roteiro de viagem personalizado para o usuário.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda em formato de roteiro com dias, horários e sugestões de atividades.
    Roteiro:""")

logistics_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens focado em logística e transporte.
    Responda à pergunta do usuário de forma direta, prática e objetiva.
    Pergunta do usuário: {question}
    Resposta:""")

local_info_prompt = ChatPromptTemplate.from_template("""
    Você é um especialista em informações locais e turismo.
    Use o contexto abaixo para responder à pergunta do usuário de forma clara e objetiva.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda de forma direta e concisa. Se a informação não estiver disponível, diga que não sabe.
    Informação:""")

translation_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de tradução e guia de frases úteis para viagens.
    Traduza a frase do usuário.
    Exemplo: "Onde fica o banheiro?" -> "Where is the bathroom?"
    Frase do usuário: {question}
    Tradução/Frase:""")

default_template = """
    Você é um assistente de viagens educado e prestativo. A pergunta do usuário não se encaixou em uma categoria específica.
    Responda de forma cortês, reconhecendo a limitação e oferecendo ajuda.
    Pergunta do usuário: {input}
    Resposta:"""
default_prompt = ChatPromptTemplate.from_template(default_template)

# Definição das Cadeias com a lógica de RAG
itinerary_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever(search_kwargs={"filter": x.get("filtros", {})}).invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | itinerary_prompt | llm | StrOutputParser())

logistics_chain = ({ "question": RunnablePassthrough() } | logistics_prompt | llm | StrOutputParser())

local_info_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever(search_kwargs={"filter": x.get("filtros", {})}).invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | local_info_prompt | llm | StrOutputParser())

translation_chain = ({ "question": RunnablePassthrough() } | translation_prompt | llm | StrOutputParser())

default_chain = ({ "input": RunnablePassthrough() } | default_prompt | llm | StrOutputParser())

In [ ]:
router_template_simples = """
    Sua tarefa é classificar uma consulta de usuário em uma das seguintes especialidades.
    Você deve retornar APENAS O NOME DA ESPECIALIDADE, sem qualquer outra palavra, explicação ou pontuação.

    Especialidades:
    - roteiro_viagem
    - logistica_transporte
    - informacao_local
    - traducao_idiomas

    Consulta: {query}
    Especialidade:
"""
router_prompt_simples = ChatPromptTemplate.from_template(router_template_simples)

router_chain = LLMChain(llm=llm_router, prompt=router_prompt_simples, output_parser=StrOutputParser())

In [27]:
test_questions = [
    "Me dê um roteiro de 3 dias em Roma.",
    "Onde fica a Escadaria Selarón?",
    "Como se diz 'por favor' em italiano?",
    "Preciso de dicas de transporte para Nova York.",
    "Que pratos típicos franceses devo experimentar?"
]

print("Testando o roteador...")
for q in test_questions:
    try:
        especialidade = router_chain.run(q).strip().lower()
        print(f"Pergunta: '{q}' -> Roteado para: '{especialidade}'")
    except Exception as e:
        print(f"Erro no roteador para a pergunta '{q}': {e}")

Testando o roteador...
Pergunta: 'Me dê um roteiro de 3 dias em Roma.' -> Roteado para: 'roteiro_viagem'
Pergunta: 'Onde fica a Escadaria Selarón?' -> Roteado para: 'informacao_local'
Pergunta: 'Como se diz 'por favor' em italiano?' -> Roteado para: 'traducao_idiomas'
Pergunta: 'Preciso de dicas de transporte para Nova York.' -> Roteado para: 'logistica_transporte'
Pergunta: 'Que pratos típicos franceses devo experimentar?' -> Roteado para: 'informacao_local'
